In [41]:
import cv2

# 画像の読み込み
background = cv2.imread("background.png")
objectA = cv2.imread("objectA.png", -1)
width, height, channels = background.shape[:3]

# 移動開始位置と移動終了位置
start_pos = (height*0.5, width*0.1)
end_pos = (height*0.5, width*0.9)

# 総フレーム数と1フレームあたりの時間(ms)
total_frames = 30
time_per_frame = 33

# 移動距離を計算
dx = (end_pos[0] - start_pos[0]) / total_frames
dy = (end_pos[1] - start_pos[1]) / total_frames

# 出力する動画の設定
fourcc = cv2.VideoWriter_fourcc(*"mp4v")
output = cv2.VideoWriter("output.mp4", fourcc, 30.0, (background.shape[1], background.shape[0]))

# Aの移動を反映した各フレームを生成して動画に書き込む
for i in range(total_frames):
    # Aを移動させた画像を生成
    x = int(start_pos[0] + i * dx)
    y = int(start_pos[1] + i * dy)
    overlay = background.copy()
    alpha = objectA[:,:,3] / 255.0
    for c in range(3):
      overlay[y:y+objectA.shape[0], x:x+objectA.shape[1], c] = (
          alpha * objectA[:, :, c] +
          (1 - alpha) * overlay[y:y+objectA.shape[0], x:x+objectA.shape[1], c]
      )
    # 動画に書き込む
    output.write(overlay)

# 出力動画を保存して、後始末
output.release()
cv2.destroyAllWindows()
